In [5]:
!pip install matplotlib-inline

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import flush_figures
plt.show = flush_figures  

from tkinter import *
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("SecureSteg - Image Steganography")
        self.master.geometry("1100x800")
        self.master.configure(bg='#f0f2f5')
        
        # Custom color scheme
        self.colors = {
            'primary': '#2c3e50',
            'secondary': '#3498db',  
            'success': '#27ae60',
            'danger': '#e74c3c',
            'light': '#ecf0f1',
            'dark': '#2c3e50'
        }

        self.host_path = None
        self.secret_path = None

        # Configure styles
        self.setup_styles()

        # Create the main widgets and layout
        self.create_widgets()

    def setup_styles(self):
        """Configure custom styles for widgets"""
        self.style = ttk.Style()
        
        # Configure main frame background
        self.style.configure('TFrame', background=self.colors['light'])
        
        # Configure label frame styles
        self.style.configure('TLabelframe', 
                           background=self.colors['light'],
                           relief='groove',
                           borderwidth=2)
        self.style.configure('TLabelframe.Label', 
                           font=('Helvetica', 12, 'bold'),
                           foreground=self.colors['primary'])
        
       
        self.style.map('TButton',
                      foreground=[('active', self.colors['secondary']),
                                  ('!active', self.colors['secondary'])],
                      background=[('active', self.colors['light']),
                                  ('!active', self.colors['light'])])
        self.style.configure('TButton',
                           font=('Arial', 10, 'bold'),
                           borderwidth=1,
                           relief='raised',
                           padding=8,
                           background=self.colors['light'],
                           foreground=self.colors['secondary'])  
        
       
        self.style.configure('TLabel',
                           background=self.colors['light'],
                           foreground=self.colors['dark'],
                           font=('Arial', 9))
        
        self.style.configure('Status.TLabel',
                           font=('Arial', 9, 'italic'),
                           background=self.colors['dark'],
                           foreground=self.colors['light'])

    def create_widgets(self):
        """Create and arrange all frames and widgets"""
        header_frame = ttk.Frame(self.master)
        header_frame.pack(fill=X, padx=20, pady=10)
        
        ttk.Label(header_frame, 
                 text="SecureSteg",
                 font=('Helvetica', 24, 'bold'),
                 foreground=self.colors['primary']).pack(side=LEFT)
        
       
        main_frame = ttk.Frame(self.master)
        main_frame.pack(fill=BOTH, expand=True, padx=20, pady=10)
    
        main_frame.rowconfigure(0, weight=1)
        main_frame.rowconfigure(1, weight=1)
        main_frame.columnconfigure(0, weight=1)

        encode_frame = ttk.LabelFrame(main_frame, text="Encode Section")
        encode_frame.grid(row=0, column=0, sticky="nsew", padx=10, pady=5)
 
        controls_frame = ttk.Frame(encode_frame)
        controls_frame.pack(fill=X, pady=10)
        
        ttk.Button(controls_frame, 
                  text="Upload Host Image", 
                  command=self.upload_host).grid(row=0, column=0, padx=5)
        ttk.Button(controls_frame, 
                  text="Upload Secret Image", 
                  command=self.upload_secret).grid(row=0, column=1, padx=5)
        ttk.Button(controls_frame, 
                  text="Encode Images", 
                  command=self.encode_images).grid(row=0, column=2, padx=5)
 
        preview_frame = ttk.Frame(encode_frame)
        preview_frame.pack(fill=BOTH, expand=True, pady=10)
        
        self.host_preview = self.create_preview_panel(preview_frame, "Host Image Preview", 0)
        self.secret_preview = self.create_preview_panel(preview_frame, "Secret Image Preview", 1)
        self.encoded_preview = self.create_preview_panel(preview_frame, "Encoded Image Preview", 2)

        decode_frame = ttk.LabelFrame(main_frame, text="Decode Section")
        decode_frame.grid(row=1, column=0, sticky="nsew", padx=10, pady=5)

        decode_controls = ttk.Frame(decode_frame)
        decode_controls.pack(fill=X, pady=10)
        
        ttk.Button(decode_controls, 
                 text="Decode Image", 
                 command=self.decode_image).pack(pady=5)

        decode_preview_frame = ttk.Frame(decode_frame)
        decode_preview_frame.pack(fill=BOTH, expand=True, pady=10)
        
        self.encoded_preview_decode = self.create_preview_panel(decode_preview_frame, "Encoded Image Input", 0)
        self.decoded_preview = self.create_preview_panel(decode_preview_frame, "Decoded Secret Image", 1)

        self.status = ttk.Label(self.master, 
                              text="Ready", 
                              style='Status.TLabel',
                              anchor=CENTER)
        self.status.pack(fill=X, side=BOTTOM, ipady=5)

    def create_preview_panel(self, parent, title, column):
        """Create a preview panel with title"""
        frame = ttk.Frame(parent)
        frame.grid(row=0, column=column, padx=10, pady=10, sticky='nsew')
        parent.columnconfigure(column, weight=1)
        
        ttk.Label(frame, 
                 text=title,
                 font=('Arial', 10, 'bold'),
                 foreground=self.colors['secondary']).pack(pady=5)
        
        container = ttk.Frame(frame, relief='sunken', borderwidth=1)
        container.pack(fill=BOTH, expand=True)
        
        label = ttk.Label(container, 
                         text="Click to upload image",
                         font=('Arial', 8),
                         foreground='#95a5a6',
                         anchor=CENTER)
        label.pack(expand=True, fill=BOTH)
        
        return label

    def upload_host(self):
        path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        if path:
            self.host_path = path
            self.update_preview(path, self.host_preview)
            self.update_status("Host image uploaded successfully!", 'success')

    def upload_secret(self):
        path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        if path:
            self.secret_path = path
            self.update_preview(path, self.secret_preview)
            self.update_status("Secret image uploaded successfully!", 'success')

    def update_preview(self, path, label_widget, size=(300, 200)):
        try:
            with Image.open(path) as img:
                img.thumbnail(size)
                photo = ImageTk.PhotoImage(img)
                label_widget.configure(image=photo, text="")
                label_widget.image = photo
                label_widget.bind("<Button-1>", lambda e: self.show_full_image(path))
        except Exception as e:
            self.update_status(f"Error loading image: {str(e)}", 'error')

    def show_full_image(self, path):
        full_view = Toplevel(self.master)
        full_view.title("Image Preview")
        full_view.configure(bg=self.colors['light'])
        try:
            with Image.open(path) as img:
                photo = ImageTk.PhotoImage(img)
                label = ttk.Label(full_view, image=photo)
                label.image = photo
                label.pack(padx=20, pady=20)
        except Exception as e:
            messagebox.showerror("Error", f"Cannot open image: {e}")

    def encode_images(self):
        if not self.host_path or not self.secret_path:
            self.update_status("Please upload both host and secret images first!", 'error')
            return

        try:
            output_path = "encoded_image.png"
            with Image.open(self.host_path) as host_img, Image.open(self.secret_path) as secret_img:
                host_img = host_img.convert("RGB")
                secret_img = secret_img.convert("RGB").resize(host_img.size)

                host_arr = np.array(host_img)
                secret_arr = np.array(secret_img)

                encoded_arr = (host_arr & 0b11111100) | (secret_arr >> 6)
                Image.fromarray(encoded_arr.astype('uint8')).save(output_path)

            self.update_preview(output_path, self.encoded_preview)
            self.update_preview(output_path, self.encoded_preview_decode)
            self.update_status("Encoding successful! Image saved as encoded_image.png", 'success')
        except Exception as e:
            self.update_status(f"Encoding failed: {str(e)}", 'error')

    def decode_image(self):
        try:
            with Image.open("encoded_image.png") as encoded_img:
                encoded_arr = np.array(encoded_img.convert("RGB"))
                decoded_arr = (encoded_arr & 0b00000011) << 6
                decoded_img = Image.fromarray(decoded_arr.astype('uint8'))

                decoded_img.thumbnail((300, 200))
                photo = ImageTk.PhotoImage(decoded_img)
                self.decoded_preview.configure(image=photo, text="")
                self.decoded_preview.image = photo

            self.update_status("Decoding successful!", 'success')
        except FileNotFoundError:
            self.update_status("No encoded image found. Encode an image first!", 'error')
        except Exception as e:
            self.update_status(f"Decoding failed: {str(e)}", 'error')

    def update_status(self, message, category='normal'):
        status_colors = {
            'normal': ('black', self.colors['dark']),
            'success': ('white', self.colors['success']),
            'error': ('white', self.colors['danger'])
        }
        fg, bg = status_colors.get(category, ('black', self.colors['dark']))
        self.status.config(text=message, foreground=fg, background=bg)
        self.master.after(5000, lambda: self.status.config(
            text="Ready", 
            foreground='white', 
            background=self.colors['dark']
        ))

if __name__ == "__main__":
    
    root = Tk()
    app = SteganographyApp(root)
    
    
    def jupyter_mainloop():
        root.update()
        root.after(100, jupyter_mainloop)
    
    root.after(100, jupyter_mainloop)
    root.mainloop()